In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import random
from math import ceil
from copy import deepcopy
random.seed(42)

2024-01-14 10:07:58.340013: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-14 10:07:58.474798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 10:07:58.474941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 10:07:58.497846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 10:07:58.552884: I tensorflow/core/platform/cpu_feature_guar

In [2]:
os.chdir('..')
os.getcwd()

'/home/abraham-pc/Documents/paid_gigs/malaria_prediction_cv'

In [3]:
def scan_directory(directory: str, extension: str) -> list:
    """Check specified directory and return list of files with
    specified extension

    Args:
        directory (str): path string to directory e.g. "./the/directory"
        extension (str): extension type to be searched for e.g. ".csv"

    Returns:
        list: strings of file names with specified extension
    """    
    files: list = []
    for filename in os.listdir(directory):
        if filename.endswith(extension):
            files.append(filename)
    return files

In [4]:
raw_path = './data/raw/SpeciesDataset/'
raw_train_path = 'train/'
raw_test_path = 'test/'
species = ['Falciparum', 'Malariae', 'Ovale', 'Vivax']

interim_path = './data/interim/'

# for split in [raw_train_path, raw_test_path]:
#     for specie in species:
#         os.makedirs(interim_path + specie, exist_ok=True)
#         for image in scan_directory(raw_path + split + specie, '.png'):
#             png_img = cv2.imread(image)
#             cv2.imwrite(interim_path + specie + '/' + image[:-4] + '.jpg', png_img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])


In [5]:
# split data into train and test by species
data_reference = pd.DataFrame(columns=['image', 'species'])

for specie in species:
    if specie == 'Falciparum':
        all_images = scan_directory(interim_path + specie, '.jpg')
        images = random.sample(all_images, ceil(np.mean([76, 31, 62])))
        full_images_path = [interim_path + specie + '/' + image for image in images]
        specie_df = pd.DataFrame({'image': full_images_path, 'species': specie})
        data_reference = pd.concat([data_reference, specie_df])
    else:
        images = scan_directory(interim_path + specie, '.jpg')
        full_images_path = [interim_path + specie + '/' + image for image in images]
        specie_df = pd.DataFrame({'image': full_images_path, 'species': specie})
        data_reference = pd.concat([data_reference, specie_df])



# Split the data into train and test sets
training_data, test_df = train_test_split(
    data_reference,
    stratify=data_reference['species'],
    test_size=0.2,
    random_state=42
)

train_df, val_df = train_test_split(
    training_data,
    stratify=training_data['species'],
    test_size=0.2,
    random_state=42
)

In [6]:
X_train = []
y_train = []
for image, specie in zip(train_df['image'], train_df['species']):
    img_array = cv2.imread(image)
    resized_array = cv2.resize(img_array, (224, 224))
    specie_num = species.index(specie)
    X_train.append(resized_array)
    y_train.append(specie_num)

X_train = np.array(X_train).reshape(-1, 224, 224, 3) / 255
y_train = np.array(y_train)


In [7]:
encoder = OneHotEncoder(sparse_output=False)
encoder = encoder.fit(y_train.reshape(-1, 1))
y_train = encoder.transform(y_train.reshape(-1, 1))

In [6]:
def getXY(dataframe, train=False):
    X = []
    y = []
    for image, specie in zip(dataframe['image'], dataframe['species']):
        img_array = cv2.imread(image)
        resized_array = cv2.resize(img_array, (224, 224))
        specie_num = species.index(specie)
        X.append(resized_array)
        y.append(specie_num)
    X = np.array(X).reshape(-1, 224, 224, 3)
    y = np.array(y)
    if train:
        encoder = OneHotEncoder(sparse_output=False)
        encoder = encoder.fit(y.reshape(-1, 1))
        joblib.dump(encoder, './src/features/encoder.joblib')
        encoder = joblib.load('./src/features/encoder.joblib')
        y = encoder.transform(y.reshape(-1, 1))
    else:
        encoder = joblib.load('./src/features/encoder.joblib')   
        y = encoder.transform(y.reshape(-1, 1))
    return X, y

In [9]:
X_trainfn, y_trainfn = getXY(train_df, train=True)

In [23]:
list(y_trainfn[0])

[1.0, 0.0, 0.0, 0.0]

In [31]:
cat_train['species'].iloc[1]

'Ovale'

In [7]:
def makeCategorical(original_df, categorical_y):
    cat_df = deepcopy(original_df)

    for i in range(len(cat_df)):
        cat_df['species'].iloc[i] = categorical_y[i]
    return cat_df

In [54]:
cat_train = makeCategorical(train_df, y_trainfn)

In [55]:
cat_train

,image,species
35,./data/interim/Falciparum/plasmodium-phone-054...,"[1.0, 0.0, 0.0, 0.0]"
24,./data/interim/Ovale/1707180816-0011-R_T.jpg,"[0.0, 0.0, 1.0, 0.0]"
63,./data/interim/Vivax/1709041080-0014-R.jpg,"[0.0, 0.0, 0.0, 1.0]"
60,./data/interim/Malariae/1401080976-0009-S_T.jpg,"[0.0, 1.0, 0.0, 0.0]"
22,./data/interim/Malariae/1312132815-0013-S.jpg,"[0.0, 1.0, 0.0, 0.0]"
...,...,...
37,./data/interim/Vivax/Thin-smear-Plasmodium-viv...,"[0.0, 0.0, 0.0, 1.0]"
57,./data/interim/Vivax/1703121298-0008-G.jpg,"[0.0, 0.0, 0.0, 1.0]"
19,./data/interim/Falciparum/plasmodium-phone-106...,"[1.0, 0.0, 0.0, 0.0]"
59,./data/interim/Vivax/1709041080-0024-R.jpg,"[0.0, 0.0, 0.0, 1.0]"


In [10]:
assert y_trainfn.all() == y_train.all()

In [11]:
y_train

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],


In [12]:
train_df.head()

,image,species
35,./data/interim/Falciparum/plasmodium-phone-054...,Falciparum
24,./data/interim/Ovale/1707180816-0011-R_T.jpg,Ovale
63,./data/interim/Vivax/1709041080-0014-R.jpg,Vivax
60,./data/interim/Malariae/1401080976-0009-S_T.jpg,Malariae
22,./data/interim/Malariae/1312132815-0013-S.jpg,Malariae


In [13]:
data_reference['species'].value_counts()

species
Vivax         76
Malariae      62
Falciparum    57
Ovale         31
Name: count, dtype: int64

In [14]:
train_df['species'].value_counts()

species
Vivax         49
Malariae      39
Falciparum    36
Ovale         20
Name: count, dtype: int64

In [15]:
test_df['species'].value_counts()

species
Vivax         15
Malariae      13
Falciparum    12
Ovale          6
Name: count, dtype: int64

In [8]:
# Create train data generator
processed_path = './data/processed/'
processed_train_path = 'train/'
processed_test_path = 'test/'
os.makedirs(processed_path + processed_train_path, exist_ok=True)
os.makedirs(processed_path + processed_test_path, exist_ok=True)
train_path = processed_path + processed_train_path
test_path = processed_path + processed_test_path


# transform data
X_train, y_train = getXY(train_df, train=True)
cat_train_df = makeCategorical(train_df, y_train)

X_val, y_val = getXY(val_df)
cat_val_df = makeCategorical(val_df, y_val)

X_test, y_test = getXY(test_df)
cat_test_df = makeCategorical(test_df, y_test)

# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
# )
# train_generator = train_datagen.flow_from_dataframe(
#     dataframe=cat_train_df,
#     directory=None,
#     x_col="image",
#     y_col="species",
#     class_mode="raw",
#     target_size=(224, 224),
#     save_to_dir=train_path,
#     batch_size=32,
#     seed=42
# )

# # Create test data generator
# test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_dataframe(
#     dataframe=cat_test_df,
#     directory=None,
#     x_col="image",
#     y_col="species",
#     class_mode=None,
#     target_size=(224, 224),
#     save_to_dir=test_path,
#     batch_size=32,
#     seed = 42
# )


In [57]:
next(train_generator)

(array([[[[0.6039728 , 0.3806811 , 0.45317537],
          [0.6069415 , 0.38659307, 0.4482473 ],
          [0.62687075, 0.4094027 , 0.46326056],
          ...,
          [0.54786575, 0.4878311 , 0.4362878 ],
          [0.5412239 , 0.47490832, 0.4528316 ],
          [0.5411788 , 0.48203126, 0.45109597]],
 
         [[0.62014776, 0.40147853, 0.4798776 ],
          [0.61036426, 0.37988362, 0.45190293],
          [0.62614346, 0.38824487, 0.45134225],
          ...,
          [0.55121607, 0.4927996 , 0.43172023],
          [0.5358131 , 0.476837  , 0.45688525],
          [0.53624505, 0.47462174, 0.4486997 ]],
 
         [[0.65236646, 0.4449696 , 0.5135523 ],
          [0.62936884, 0.38845253, 0.46136743],
          [0.6178141 , 0.37628993, 0.44827577],
          ...,
          [0.5607214 , 0.4900243 , 0.44080755],
          [0.5403916 , 0.48294646, 0.46256813],
          [0.52715003, 0.47586858, 0.45103714]],
 
         ...,
 
         [[0.7420572 , 0.6139939 , 0.59145355],
          [0.72128

In [49]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2)

datagen.fit(X_train)


In [60]:
datagen.flow(X_train, batch_size=1).next()[0].shape

(224, 224, 3)

In [28]:
def transform_data(array, modify = False):
    if modify:
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2)
    else:
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

    datagen.fit(array)

    new_array = []

    while len(new_array) < len(array):
        new_array.extend(datagen.flow(array, shuffle=False).next())

    new_array = np.array(new_array)
    return new_array

In [29]:
new_X_train = transform_data(X_train)

In [26]:
X_train[0]

array([[[0.43921569, 0.47058824, 0.70588235],
        [0.44705882, 0.47843137, 0.71372549],
        [0.44313725, 0.48235294, 0.71764706],
        ...,
        [0.51764706, 0.53333333, 0.69803922],
        [0.51764706, 0.53333333, 0.68627451],
        [0.51372549, 0.53333333, 0.68627451]],

       [[0.44313725, 0.4745098 , 0.70980392],
        [0.44313725, 0.4745098 , 0.70980392],
        [0.44313725, 0.48235294, 0.71764706],
        ...,
        [0.51372549, 0.52941176, 0.69019608],
        [0.51372549, 0.52941176, 0.69019608],
        [0.52156863, 0.54117647, 0.69411765]],

       [[0.45098039, 0.48627451, 0.72156863],
        [0.44313725, 0.47843137, 0.71372549],
        [0.46666667, 0.49803922, 0.73333333],
        ...,
        [0.5254902 , 0.54117647, 0.70196078],
        [0.51764706, 0.53333333, 0.69411765],
        [0.51372549, 0.52941176, 0.69411765]],

       ...,

       [[0.53333333, 0.60784314, 0.84705882],
        [0.5372549 , 0.61176471, 0.85098039],
        [0.5372549 , 0

In [30]:
new_X_train[0]

array([[[0.00172241, 0.00184544, 0.00276817],
        [0.00175317, 0.0018762 , 0.00279892],
        [0.00173779, 0.00189158, 0.0028143 ],
        ...,
        [0.00202999, 0.0020915 , 0.00273741],
        [0.00202999, 0.0020915 , 0.00269127],
        [0.00201461, 0.0020915 , 0.00269127]],

       [[0.00173779, 0.00186082, 0.00278355],
        [0.00173779, 0.00186082, 0.00278355],
        [0.00173779, 0.00189158, 0.0028143 ],
        ...,
        [0.00201461, 0.00207612, 0.00270665],
        [0.00201461, 0.00207612, 0.00270665],
        [0.00204537, 0.00212226, 0.00272203]],

       [[0.00176855, 0.00190696, 0.00282968],
        [0.00173779, 0.0018762 , 0.00279892],
        [0.00183007, 0.0019531 , 0.00287582],
        ...,
        [0.00206075, 0.00212226, 0.00275279],
        [0.00202999, 0.0020915 , 0.00272203],
        [0.00201461, 0.00207612, 0.00272203]],

       ...,

       [[0.0020915 , 0.0023837 , 0.0033218 ],
        [0.00210688, 0.00239908, 0.00333718],
        [0.00210688, 0

In [21]:
assert X_train[0][0][0][0] != new_X_train[0][0][0][0]